In [54]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import math
import numdifftools as nd
import pandas as pd
import pymc3 as pm

from tqdm import tqdm
from sklearn import preprocessing
from numpy.linalg import multi_dot
import scipy
from scipy import stats
from scipy.stats import invgamma
from scipy.special import gamma
from scipy.special import digamma
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import levy_stable

In [ ]:
# GENERATE DATA FROM STABLE DISTRI

In [ ]:
# CALCULATE SUMMARY STATS

## LOAD DATASET

In [55]:
# Generate actual data
actual_data = levy_stable.rvs(1.5, 0.5, scale = 1, loc = 0,  size=200)

# Calculate summary stats of actual data
stability = (np.percentile(actual_data, 95) - np.percentile(actual_data, 5)) / (np.percentile(actual_data, 75) - np.percentile(actual_data, 25))
skewness = (np.percentile(actual_data, 95) + np.percentile(actual_data, 5) - 2 * np.percentile(actual_data, 50)) / (np.percentile(actual_data, 95) - np.percentile(actual_data, 5))
scale = (np.percentile(actual_data, 75) - np.percentile(actual_data, 25)) / 1
loc = np.mean(actual_data)

actual_summary_statistics = np.array([stability, skewness, scale, loc])
actual_summary_statistics

array([ 3.17408065,  0.19437762,  1.9275681 , -0.12839691])

## FIND THE BEST THETA (COEFFICIENTS) USING VB

In [93]:
class GVB:
    def __init__(self, samples, actual_summary_statistics, learning_rate, threshold, l_threshold, adaptive_lr_1, adaptive_lr_2, t_w, Patience):
        self.samples = samples
        self.actual_summary_statistics = actual_summary_statistics
        self.num_datasets = 100 # number of datasets
        self.num_coeffs = np.shape(actual_summary_statistics)[0] # number of coeffs
        self.lambda_dim = self.num_coeffs + int((self.num_coeffs * (self.num_coeffs + 1)) / 2)
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.l_threshold = l_threshold
        self.adaptive_lr_1 = adaptive_lr_1
        self.adaptive_lr_2 = adaptive_lr_2
        self.t_w = t_w
        self.Patience = Patience

    def summary_statistics(self, theta):
        n_datasets = []
        n_stability = []
        n_skewness = []
        n_scale = []
        n_loc = []
        for i in range(self.num_datasets):
            n_datasets.append(levy_stable.rvs(theta[0], theta[1], scale = theta[2], loc = theta[3], size=200))
            n_stability.append((np.percentile(n_datasets[i], 95) - np.percentile(n_datasets[i], 5)) / (np.percentile(n_datasets[i], 75) - np.percentile(n_datasets[i], 25)))
            n_skewness.append((np.percentile(n_datasets[i], 95) + np.percentile(n_datasets[i], 5) - 2 * np.percentile(n_datasets[i], 50)) / (np.percentile(n_datasets[i], 95) - np.percentile(n_datasets[i], 5)))
            n_scale.append((np.percentile(n_datasets[i], 75) - np.percentile(n_datasets[i], 25)) / theta[2])
            n_loc.append(np.mean(n_datasets[i]))

        summary_statistics = np.array([n_stability, n_skewness, n_scale, n_loc])
        sample_mean = np.mean(summary_statistics, axis = 1)
        sample_variance = np.cov(summary_statistics)

        return sample_mean, sample_variance

    # def prior(self, theta): 
    #     log_prior = - (self.num_coeffs/2) * np.log(2 * math.pi) - (self.num_coeffs/2) * np.log(100) - np.dot(theta.T, theta) / (2 * 100)
    #     return log_prior

    def prior(self, theta): 
        log_prior = multivariate_normal.logpdf(theta, cov= 100 * np.identity(self.num_coeffs))
        
        return log_prior

    def unbiased_log_likelihood(self, theta):
        sample_mean = self.summary_statistics(theta)[0]
        sample_variance = self.summary_statistics(theta)[1]
        diff_mean_s = self.actual_summary_statistics - sample_mean
        part1 = diff_mean_s.T @ np.linalg.inv(sample_variance) @ diff_mean_s
        u_est_log_likelihood = -1/2 * np.log(np.linalg.det(sample_variance)) - (self.num_datasets - self.num_coeffs - 2) / (2 * (self.num_datasets-1)) * part1
        return u_est_log_likelihood

    # def log_q(self, theta, mu, l):
    #     log_q = - (self.num_coeffs/2) * np.log(2 * math.pi) - (1/2) * np.log(np.linalg.det(l)) - (1/2) * np.matmul(np.matmul(np.matmul(theta - mu, l), l.T), (theta - mu).T)
    #     return log_q

    def log_q(self, theta, mu, l):
        log_q = multivariate_normal.logpdf(theta, mean = mu, cov= np.linalg.inv(l @ l.T))
        return log_q

    def gradient_log_q(self, theta, mu, l): #indep theta
        gradient_log_q_mu = l @ l.T @ (theta - mu)
        gradient_log_q_l = (np.diag(1 / np.diag(l)) - ((theta - mu) @ (theta - mu).T * l)).T[np.triu_indices(self.num_coeffs)] #use * because matmul gives scalar 
        gradient_log_q = np.array([gradient_log_q_mu, gradient_log_q_l], dtype=object)
        return gradient_log_q

    def control_variates(self, Flat_grad_log_q, Flat_grad_lb):
        c = []
        stacked_gradient_lb = np.stack(Flat_grad_lb)
        stacked_gradient_log_q = np.stack(Flat_grad_log_q)
        for i in range(self.lambda_dim):
            sample_cov = np.cov((stacked_gradient_lb[:, i], stacked_gradient_log_q[:, i]))
            c_i = sample_cov[0, 1] / sample_cov[1, 1]
            c.append(c_i)
        c_mu = np.array(c[0:self.num_coeffs])
        c_vech_l = np.array(c[self.num_coeffs:])
        return np.array([c_mu, c_vech_l], dtype = object)

    def vb_posterior(self, stop):
        # Initialize mu_0, L_0
        #mu_0 = np.array([np.log((self.actual_summary_statistics[0]-1.1)/(2-self.actual_summary_statistics[0])), np.log((1+self.actual_summary_statistics[1])/(1-self.actual_summary_statistics[1])), np.log(self.actual_summary_statistics[2]), self.actual_summary_statistics[3]])
        #mu_0 = np.array([1.5, 0.5, 3, 0])
        #mu_0 = np.array([self.actual_summary_statistics[0], self.actual_summary_statistics[1], self.actual_summary_statistics[2], self.actual_summary_statistics[3]])
        mu_0 = np.array([0] * self.num_coeffs)
        l_0 = np.identity(self.num_coeffs) * 10
        Sigma_0_inv = l_0 @ l_0.T
        Sigma_0 = np.linalg.inv(Sigma_0_inv)
        l_0_inv = np.linalg.inv(l_0)
        ### Change ways to get vech(l0)
        vech_l0 = Sigma_0.T[np.triu_indices(self.num_coeffs)]

        lambda_0 = np.array([mu_0, vech_l0], dtype = object)
        lambda_q = lambda_0
        # List of Lambda
        Lambda = [lambda_0]
        # List of theta
        Theta = []
        Theta_t = []
        # List of calculations of LB
        LB_all = []
        LB_Smoothed = []
        patience = 0
        # List of flattened gradients
        Flattened_gradient_lb = []
        Flattened_gradient_log_q = []
        for t in tqdm(range(stop)):
            if t == 0:        
                # Draw samples of theta from  variational q
                # List of gradients
                Gradient_lb_init = []
                lb_0 = []
                theta_0_samples = multivariate_normal.rvs(mean = mu_0, cov = Sigma_0, size = self.samples)
                for s in range(self.samples):
                    # True params have been reparam into epsilon
                    theta_tilde_0 = theta_0_samples[s]
                    alpha_0 = (2 * np.exp(theta_tilde_0[0]) + 1.1) / (1 + np.exp(theta_tilde_0[0]))
                    beta_0 = (np.exp(theta_tilde_0[1]) - 1) / (np.exp(theta_tilde_0[1]) + 1)
                    gamma_0 = np.exp(theta_tilde_0[2])
                    delta_0 = theta_tilde_0[3]
                    theta_0 = np.array([alpha_0, beta_0, gamma_0, delta_0])
                    
                    Theta_t.append(theta_0)
                    # Find gradient of LB
                    h_lambda_init = self.prior(theta_tilde_0) + self.unbiased_log_likelihood(theta_0) - self.log_q(theta_tilde_0, mu_0, l_0)
                    gradient_lb_init = self.gradient_log_q(theta_tilde_0, mu_0, l_0) * (h_lambda_init)
                    Gradient_lb_init.append(gradient_lb_init)
                    # Calculate control variates
                    flattened_gradient_log_q = np.concatenate((self.gradient_log_q(theta_tilde_0, mu_0, l_0)[0], self.gradient_log_q(theta_tilde_0, mu_0, l_0)[1]), axis = None)
                    Flattened_gradient_log_q.append(flattened_gradient_log_q)
                    flattened_gradient_lb = np.concatenate((gradient_lb_init[0], gradient_lb_init[1]), axis = None)
                    Flattened_gradient_lb.append(flattened_gradient_lb)
                    # Calculate lower bound
                    lb_0.append(h_lambda_init)
                # Calculate control variates using all samples
                c = self.control_variates(Flattened_gradient_log_q, Flattened_gradient_lb)
                # Update lambda_q
                self.g_init = np.mean(Gradient_lb_init, axis = 0)
                # Gradient clipping
                if np.linalg.norm(np.concatenate(self.g_init, axis = None)) > self.l_threshold:
                    self.g_init = self.l_threshold * self.g_init / np.linalg.norm(np.concatenate(self.g_init, axis = None))
                self.v_init = self.g_init ** 2
                # Calculate lower bound
                LB_all.append(np.mean(lb_0))
                Theta.append(Theta_t)
            if t > 0:
                # From lambda_q find mu_q and L_q
                mu_q = lambda_q[0]

                ### Change ways to convert from vech_l0 to l0
                l_q = np.zeros((self.num_coeffs, self.num_coeffs))
                l_q[:, 0] = lambda_q[1][0:self.num_coeffs]
                l_q[1:self.num_coeffs, 1] = lambda_q[1][4:7]
                l_q[2:self.num_coeffs, 2] = lambda_q[1][7:9]
                l_q[3:self.num_coeffs, 3] = lambda_q[1][9:10]

                Sigma_q_inv = l_q @ l_q.T
                Sigma_q =  np.linalg.inv(Sigma_q_inv)
                l_q_inv =  np.linalg.inv(l_q)
                # List of gradients
                Gradient_lb = []
                lb_t = []
                theta_q_samples = multivariate_normal.rvs(mean = mu_q, cov = Sigma_q, size = self.samples)
                for s in range(self.samples):
                    theta_tilde_q = theta_q_samples[s]
                    # Calculate theta from mu, l (lambda)
                    alpha_q = (2 * np.exp(theta_tilde_q[0]) + 1.1) / (1 + np.exp(theta_tilde_q[0]))
                    beta_q = (np.exp(theta_tilde_q[1]) - 1) / (np.exp(theta_tilde_q[1]) + 1)
                    gamma_q = np.exp(theta_tilde_q[2])
                    delta_q = theta_tilde_q[3]
                    theta_q = np.array([alpha_q, beta_q, gamma_q, delta_q])
                    
                    Theta_t.append(theta_q)
                    # Find gradient of LB
                    h_lambda = self.prior(theta_tilde_q) + self.unbiased_log_likelihood(theta_q) - self.log_q(theta_tilde_q, mu_q, l_q)
                    # Find gradient of LB
                    gradient_lb = self.gradient_log_q(theta_tilde_q, mu_q, l_q) * (h_lambda - c)
                    Gradient_lb.append(gradient_lb)
                    # Calculate control variates
                    Flattened_gradient_log_q[s] = np.concatenate((self.gradient_log_q(theta_tilde_q, mu_q, l_q)[0], self.gradient_log_q(theta_tilde_q, mu_q, l_q)[1]), axis = None)
                    Flattened_gradient_lb[s] = np.concatenate((gradient_lb[0], gradient_lb[1]), axis = None)
                    # Calc lower bound estimate
                    lb_t.append(h_lambda)
                # Update control variates
                c = self.control_variates(Flattened_gradient_log_q, Flattened_gradient_lb)
                # Calc gradient of h
                g_t = np.mean(Gradient_lb, axis = 0)
                # Gradient clipping
                if np.linalg.norm(np.concatenate(g_t, axis = None)) > self.l_threshold:
                    g_t = self.l_threshold * g_t / np.linalg.norm(np.concatenate(g_t, axis = None))
                v_t = g_t ** 2

                Theta.append(Theta_t)
                #---- Update lambda
                self.g_init = self.adaptive_lr_1 * self.g_init + (1 - self.adaptive_lr_1) * g_t
                self.v_init = self.adaptive_lr_2 * self.v_init + (1 - self.adaptive_lr_2) * v_t

                if t >= self.threshold:
                    update_t = self.learning_rate * self.threshold / t
                else:
                    update_t = self.learning_rate

                lambda_q = lambda_q + update_t * self.g_init / (self.v_init ** 0.5)
                Lambda.append(lambda_q)
                # Calculate lower bound
                LB_all.append(np.mean(lb_t))

                if t >= self.t_w:
                    LB_smoothed = np.mean(LB_all[t - self.t_w + 1 : t])
                    print(LB_smoothed)
                    LB_Smoothed.append(LB_smoothed)
                    if LB_smoothed < max(LB_Smoothed):
                        patience += 1
                        if patience > self.Patience:
                            print("Stop at", t)
                            break

                    best_LB_index = np.argmax(LB_Smoothed) + self.t_w - 1
                    best_Lambda = Lambda[best_LB_index]
                    best_Theta = Theta[best_LB_index]
                    # best_Mu = best_Lambda[0]
                    # best_L = np.zeros((self.num_coeffs, self.num_coeffs))
                    # for i in (np.arange(self.num_coeffs)):
                    #     index_from_tril = int((i+1) * (i+2) / 2 - 1)
                    #     best_L[i][i] = best_Lambda[1][index_from_tril] 

        return LB_all, LB_Smoothed, best_LB_index, best_Lambda, best_Theta

## RUN VB AND PRINT OUT VARIATIONAL PARAMS

In [57]:
# Set hyperparameters
stop = 5000

In [94]:
vb = GVB(200, actual_summary_statistics, 0.001, 2500, 100, 0.9, 0.9, 50, 50)
LB_estimate, smoothed_LB_estimate, best_LB_idx, best_lambda, best_theta = vb.vb_posterior(stop)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
LB_estimate

In [ ]:
best_lambda

In [ ]:
best_theta

## PLOT DENSITY PLOT OF ALL COEFFS


In [ ]:
lb_df = pd.DataFrame(np.array(LB_estimate))
plt.figure()
lb_df.plot(title = 'Lower Bound Estimate', legend = False)

In [ ]:
lb_df = pd.DataFrame(np.array(smoothed_LB_estimate))
plt.figure()
lb_df.plot(title = 'Smoothed Lower Bound Estimate', legend = False)